In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# import libraries to use
import pyrealsense2 as rs
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from cv2 import VideoWriter, VideoWriter_fourcc
import pandas as pd
import torch
from scipy import signal
from tqdm import tqdm

In [3]:
# create CNN models for face detection and landmark localization | send the models to CUDA is available | load model weights || this procedure only happens once 
from face_alignment.utils import *
from face_alignment import api as face_alignment
from face_alignment.models import FAN
from face_alignment.detection.sfd import sfd_detector

def load_weights(model, filename):
    sd = torch.load(filename, map_location=lambda storage, loc: storage)
    names = set(model.state_dict().keys())
    for n in list(sd.keys()): 
        if n not in names and n+'_raw' in names:
            if n+'_raw' not in sd: sd[n+'_raw'] = sd[n]
            del sd[n]
    model.load_state_dict(sd)

face_alignment_model = r"./models/2DFAN4-11f355bf06.pth.tar"
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
#Face alignement
network_size = 4
face_alignment_net = FAN(network_size)
load_weights(face_alignment_net,face_alignment_model)
face_alignment_net.to(device)
face_alignment_net.eval()
#face detection 
face_detector_model = r"./models/s3fd-619a316812.pth"
face_detection_net = sfd_detector.SFDDetector(device=device, path_to_detector=face_detector_model, verbose=False)

In [4]:
def get_color_from_bag(BAG_File):

    pipeline = rs.pipeline()
    config = rs.config()

    rs.config.enable_device_from_file(config, BAG_File, repeat_playback=False)
    print('here1')
    config.enable_all_streams()
    profile = pipeline.start(config)

    # create alignment object
    align_to = rs.stream.color
    align = rs.align(align_to)

    # inform the device that this is not live streaming from camera
    playback = profile.get_device().as_playback()
    playback.set_real_time(False)
    duration = playback.get_duration()

    true_frame_number = []
    frame_number = []
    time_st = []

    num_frame = 0


    Color_Frames = []#{}

    try:
        while True:
            frames = pipeline.wait_for_frames(100)  #get frame from file 

            this_frame = frames.get_frame_number()  #get frame number 

            if (num_frame != 0) and (true_frame_number[-1] == this_frame): #verify that frame number is not repeated 
                #if frame number is repeated then replace the stored information 
                aligned_frames = align.process(frames)

                #take color and depth from frame, if any to these is not available then skip the frame
                aligned_depth = aligned_frames.get_depth_frame()
                aligned_color = aligned_frames.get_color_frame()

                # validate that both frames are available
                if not aligned_depth or not aligned_color:
                    continue

                time_stamp = frames.get_timestamp()
                true_frame_number[-1] = frames.get_frame_number()
                time_st[-1] = time_stamp 

                # transform to np array

                color_data = np.asanyarray(aligned_color.as_frame().get_data(), dtype=np.int)
                #depth_data = np.asanyarray(aligned_depth.as_frame().get_data(), dtype=np.int)
                # adjust depth data in meters
                #depth_data *= depth_scale

                Color_Frames[-1] = color_data

            else:
                #if frame number is not repeated then append the stored information 
                aligned_frames = align.process(frames)

                #take color and depth from frame, if any to these is not available then skip the frame
                aligned_depth = aligned_frames.get_depth_frame()
                aligned_color = aligned_frames.get_color_frame()

                # validate that both frames are available
                if not aligned_depth or not aligned_color:
                    continue

                time_stamp = frames.get_timestamp()
                true_frame_number.append(frames.get_frame_number())
                time_st.append(time_stamp )

                # transform to np array

                color_data = np.asanyarray(aligned_color.as_frame().get_data(), dtype=np.int)
                #depth_data = np.asanyarray(aligned_depth.as_frame().get_data(), dtype=np.int)
                # adjust depth data in meters
                #depth_data *= depth_scale

                Color_Frames.append(color_data)
                #Depth_Frames.append(depth_data

                frame_number.append(num_frame)
                num_frame += 1
    
    except RuntimeError:
        pass
    finally:
        pipeline.stop()
    print('here2')

    duration_movie = duration.total_seconds()
    FPS = num_frame/duration_movie
    height, width,_ =  Color_Frames[0].shape
    print(FPS)
    color_file = BAG_File[:-4]+'_color.mp4'
    print(color_file)
    video = VideoWriter(color_file, 0, int(FPS), (width,height))

    for k in range(num_frame):
        frame_to_save = Color_Frames[k].astype('uint8')
        video.write(frame_to_save)

    video.release()    

    print('here4')
    cvs_frame_info = BAG_File[:-4]+'_frameInfoColor.csv'
    df_cols = ['Actual_Frame_Number', 'Frame_Time_Stamp', 'Frame_Number_in_Video']
    df = pd.DataFrame(columns=df_cols)
    df['Actual_Frame_Number'] = true_frame_number
    df['Frame_Time_Stamp'] = (np.array(time_st)-time_st[0])/1000
    df['Frame_Number_in_Video'] = frame_number

    df.to_csv(cvs_frame_info)
    print(cvs_frame_info)
    

def get_landmarks(BAG_File, face_detection_net, face_alignment_net):
    #load information from the video 
    video_frame_info = pd.read_csv(BAG_File[:-4]+'_frameInfoColor.csv')
    true_frame_number = video_frame_info['Actual_Frame_Number'].values
    time_st = video_frame_info['Frame_Time_Stamp'].values
    
    
    localize_face = 0
    # localize_face = 0 -> Face is localized at a single frame in the video (the middle frame)
    # localize_face = -1 -> Face is localized at each frame of the video
    # localize_face = n -> face is localized every n frames 

    # we will start by localizing the face in the middel of the video, if additional information is needed 
    # then will be added as required
    color_file = BAG_File[:-4]+'_color.mp4'
    video_handler = cv2.VideoCapture(color_file)  # read the video
    num_frames = int(video_handler.get(cv2.CAP_PROP_FRAME_COUNT))
    video_fps = int(video_handler.get(cv2.CAP_PROP_FPS))
    video_handler.set(cv2.CAP_PROP_POS_FRAMES, num_frames//2)

    success, image = video_handler.read()

    if success: 
        detected_faces = face_detection_net.detect_from_image(image)
        for i, d in enumerate(detected_faces):
            center = torch.FloatTensor(
                [d[2] - (d[2] - d[0]) / 2.0, d[3] - (d[3] - d[1]) / 2.0])
            center[1] = center[1] - (d[3] - d[1]) * 0.12
            scale = (d[2] - d[0] + d[3] - d[1]) / face_detection_net.reference_scale
    video_handler.release()

    # at this point we have the position of the face in the mid frame. Let's use that info

    #create a dataframe that will store all the information 
    df_cols = ["BAG_Frame_number","Video_Frame_number", "bbox_top_x", "bbox_top_y", "bbox_bottom_x", "bbox_bottom_y"]
    for i in range(0,68):
        num=str(i)
        xx = 'landmark_'+num+'_x'
        yy = 'landmark_'+num+'_y'
        df_cols.append(xx)
        df_cols.append(yy)

    LandmarkDataFrame = pd.DataFrame(columns = df_cols)

    # re-position the video handler at the first frame and start going frame by frame
    video_handler = cv2.VideoCapture(color_file)  # read the video
    k = 0
    success = True
    while success:
        success, image = video_handler.read()
        if success:

            if localize_face == 0:
                #do not localize the face, use previous info 
                pass 
            elif localize_face == -1 :
                #localize the face at each frame, upd
                update_detected_faces = face_detection_net.detect_from_image(image)
                for i, d in enumerate(update_detected_face):

                    if d[4]>=0.8:
                        #do we trust the face localizer, if yes (>0.8) then update the bounding box, 
                        # if not (<0.8) don't update the bounding box
                        detected_faces = update_detected_face
                        center = torch.FloatTensor(
                            [d[2] - (d[2] - d[0]) / 2.0, d[3] - (d[3] - d[1]) / 2.0])
                        center[1] = center[1] - (d[3] - d[1]) * 0.12
                        scale = (d[2] - d[0] + d[3] - d[1]) / face_detection_net.reference_scale

            else:
                #only update every n frames
                if (k+1)%localize_face == 0:
                    update_detected_face = face_detection_net.detect_from_image(image)
                    for i, d in enumerate(detected_faces):

                        if d[4]>=0.8:
                            #do we trust the face localizer, if yes (>0.8) then update the bounding box, 
                            # if not (<0.8) don't update the bounding box
                            detected_faces = update_detected_face
                            center = torch.FloatTensor(
                                [d[2] - (d[2] - d[0]) / 2.0, d[3] - (d[3] - d[1]) / 2.0])
                            center[1] = center[1] - (d[3] - d[1]) * 0.12
                            scale = (d[2] - d[0] + d[3] - d[1]) / face_detection_net.reference_scale


            inp = crop(image, center, scale)
            inp = torch.from_numpy(inp.transpose(
                        (2, 0, 1))).float()
            inp = inp.to(device)
            inp.div_(255).unsqueeze_(0)

            out = face_alignment_net(inp)[-1].detach() #[-1] is to get the output of the last hourglass block
            out = out.cpu()
            pts, pts_img = get_preds_fromhm(out, center, scale)

            pts_img = pts_img.view(68, 2)



            # Store everything in a dataframe
            datus = []
            datus.append(true_frame_number[int(k)])  #frame number provided by the .bag file 
            datus.append(int(k)+1)  # frame number in the color_only video 

            datus.append(detected_faces[0][0])  #top
            datus.append(detected_faces[0][1])  #left
            datus.append(detected_faces[0][2])  #bottom
            datus.append(detected_faces[0][3])  #right

            all_landmarks = pts_img.numpy()
            for x,y in all_landmarks:
                datus.append(x), datus.append(y)  #x and y position of each landmark

            LandmarkDataFrame = LandmarkDataFrame.append(pd.Series(datus,index = df_cols), 
                                   ignore_index = True)

            k +=1 


    #add time to landmarks 
    LandmarkDataFrame.insert(loc=1, column='Time_Stamp (s)', value=time_st)

    landmark_file = BAG_File[:-4]+'_landmarks.csv'
    print(landmark_file)
    LandmarkDataFrame.to_csv(landmark_file) 

def smooth_landmarks(BAG_File):
    #Smooth landmarks positions and generate a new video showing landmarks
    LandmarkDataFrame = pd.read_csv(BAG_File[:-4]+'_landmarks.csv', index_col=0)
    windowlength=5
    for i in range(68):
        num=str(i)
        xx = LandmarkDataFrame['landmark_'+num+'_x'].values
        xx_med = signal.medfilt(xx,kernel_size=windowlength)

        yy = LandmarkDataFrame['landmark_'+num+'_y'].values
        yy_med = signal.medfilt(yy,kernel_size=windowlength)  

        LandmarkDataFrame['landmark_'+num+'_x'] = xx_med
        LandmarkDataFrame['landmark_'+num+'_y'] = yy_med

    landmark_file = BAG_File[:-4]+'_landmarksFiltered.csv'
    LandmarkDataFrame.to_csv(landmark_file)    

    color_file = BAG_File[:-4]+'_color.mp4'
    video_handler = cv2.VideoCapture(color_file)  # read the video
    FPS = video_handler.get(cv2.CAP_PROP_FPS)
    width = video_handler.get(3)   # float
    height = video_handler.get(4) # float
    
    color_file_landmark = BAG_File[:-4]+'_colorlandmark.mp4'
    video = VideoWriter(color_file_landmark, -1, int(FPS), (int(width),int(height)))

    success = True 
    k=0
    for k in range(len(LandmarkDataFrame)):
        success, image = video_handler.read()

        frame_number=k+1
        frame_information = LandmarkDataFrame.loc[LandmarkDataFrame['Video_Frame_number'] == frame_number].values
        shape = np.array([frame_information[0][7:]])
        shape = np.reshape(shape.astype(np.int), (-1, 2))
        bb = np.array([frame_information[0][3:7]])

        
        for j, (x, y) in enumerate(shape):
            if x is np.NaN:
                continue
            elif j<17:
                continue
            else:
                cv2.rectangle(image, (int(bb[0][0]), int(bb[0][1])), (int(bb[0][2]), int(bb[0][3])), (255,0,0), 1)
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)


        frame_to_save = image
        video.write(frame_to_save)
        k +=1

    video.release()

    
def get_3d_data(BAG_File):
    #load landmakrs information
    DF_landmarks = pd.read_csv(BAG_File[:-4]+'_landmarksFiltered.csv', index_col=0)

    #create dataframe to store information about 3d position of landmarks
    df_cols_p1 = ["BAG_Frame_number","Video_Frame_number",]
    for i in range(0,68):
        num=str(i)
        xx = 'landmark_'+num
        df_cols_p1.append(xx)
        df_cols_p1.append(xx)
        df_cols_p1.append(xx)

    df_cols_p2 = ["",""]
    for i in range(0,68):
        df_cols_p2.append("x")
        df_cols_p2.append("y")
        df_cols_p2.append("z")

    header = [np.array(df_cols_p1), 
    np.array(df_cols_p2)] 

    DF_3dpositions= pd.DataFrame(columns = header)

    # start the process of extracting the video information for each video
    pipeline = rs.pipeline()
    config = rs.config()

    rs.config.enable_device_from_file(config, BAG_File, repeat_playback=False)

    config.enable_all_streams()
    profile = pipeline.start(config)

    # create alignment object
    align_to = rs.stream.color
    align = rs.align(align_to)

    # Getting the depth sensor's depth scale (see rs-align example for explanation)
    depth_sensor = profile.get_device().first_depth_sensor()
    depth_scale = depth_sensor.get_depth_scale()

    # inform the device that this is not live streaming from camera
    playback = profile.get_device().as_playback()
    playback.set_real_time(False)
    duration = playback.get_duration()

    #fill holes in the depth information (based on this example: https://nbviewer.jupyter.org/github/IntelRealSense/librealsense/blob/jupyter/notebooks/depth_filters.ipynb)
    spatial = rs.spatial_filter()
    spatial.set_option(rs.option.filter_magnitude, 2)
    spatial.set_option(rs.option.filter_smooth_alpha, 0.5)
    spatial.set_option(rs.option.filter_smooth_delta, 20)
    spatial.set_option(rs.option.holes_fill, 3)

    true_frame_number = []
    frame_number = []
    time_st = []

    num_frame = 0

    try:
        while True:
            frames = pipeline.wait_for_frames(100)

            this_frame = frames.get_frame_number()  #get frame number

            #verify that we have landmarks for this particular frame 
            landmarks = DF_landmarks.loc[DF_landmarks['BAG_Frame_number'] == this_frame].values

            #if there are not landmakrs then just ignore the frame

            if len(landmarks)>0 : 
                #continue only if landmarks for the frame are avaliable
                landmarks = landmarks[0][7:]
                landmarks = landmarks.astype('float').reshape(-1, 2)

                if (num_frame != 0) and (true_frame_number[-1] == this_frame): #verify that frame number is not repeated 
                    #frame is repeated
                    aligned_frames = align.process(frames)

                    #take color and depth from frame, if any to these is not available then skip the frame
                    aligned_depth = aligned_frames.get_depth_frame()
                    aligned_color = aligned_frames.get_color_frame()

                    # validate that both frames are available
                    if not aligned_depth or not aligned_color:
                        continue

                    time_stamp = frames.get_timestamp()
                    true_frame_number[-1] = frames.get_frame_number()
                    time_st[-1] = time_stamp
                    frame_number[-1] = num_frame

                    # Intrinsics & Extrinsics
                    depth_intrin = aligned_depth.profile.as_video_stream_profile().intrinsics
                    color_intrin = aligned_depth.profile.as_video_stream_profile().intrinsics
                    depth_to_color_extrin = aligned_depth.profile.get_extrinsics_to(aligned_color.profile)

                    aligned_filtered_depth = spatial.process(aligned_depth)
                    depth_frame_array = np.asanyarray(aligned_filtered_depth.get_data())
                    depth_frame_array = depth_frame_array*depth_scale

                    coords = []
                    coords.append(frames.get_frame_number())
                    coords.append(int(num_frame)+1)

                    for (c,r) in landmarks:  #landmarks provide the x,y position of each landmark. x are columns and y are rows in the figure
                        #depth_value = depth_frame.get_distance(int(c),int(r))
                        #x,y,z = rs.rs2_deproject_pixel_to_point(depth_intrin, [int(c), int(r)], depth_value)
                        depth_value = depth_frame_array[int(r),int(c)]
                        z = depth_value
                        x = z*((c-depth_intrin.ppx)/depth_intrin.fx)
                        y = z*((r-depth_intrin.ppy)/depth_intrin.fy)                    
                        coords.append(x),coords.append(y),coords.append(z)

                    #DF_3dpositions = DF_3dpositions.append(pd.Series(coords,index = header), ignore_index = True)
                    DF_3dpositions.iloc[-1] =  pd.Series(coords,index = header)
                else:
                    aligned_frames = align.process(frames)

                    #take color and depth from frame, if any to these is not available then skip the frame
                    aligned_depth = aligned_frames.get_depth_frame()
                    aligned_color = aligned_frames.get_color_frame()

                    # validate that both frames are available
                    if not aligned_depth or not aligned_color:
                        continue

                    time_stamp = frames.get_timestamp()
                    true_frame_number.append(frames.get_frame_number())
                    time_st.append(time_stamp)
                    frame_number.append(num_frame)

                    # Intrinsics & Extrinsics
                    depth_intrin = aligned_depth.profile.as_video_stream_profile().intrinsics
                    color_intrin = aligned_depth.profile.as_video_stream_profile().intrinsics
                    depth_to_color_extrin = aligned_depth.profile.get_extrinsics_to(aligned_color.profile)

                    aligned_filtered_depth = spatial.process(aligned_depth)
                    depth_frame_array = np.asanyarray(aligned_filtered_depth.as_frame().get_data())
                    depth_frame_array = depth_frame_array*depth_scale

                    coords = []
                    coords.append(frames.get_frame_number())
                    coords.append(int(num_frame)+1)

                    for (c,r) in landmarks:  #landmarks provide the x,y position of each landmark. x are columns and y are rows in the figure
                        #depth_value = depth_frame.get_distance(int(c),int(r))
                        #x,y,z = rs.rs2_deproject_pixel_to_point(depth_intrin, [int(c), int(r)], depth_value)
                        depth_value = depth_frame_array[int(r),int(c)]
                        z = depth_value
                        x = z*((c-depth_intrin.ppx)/depth_intrin.fx)
                        y = z*((r-depth_intrin.ppy)/depth_intrin.fy)                    
                        coords.append(x),coords.append(y),coords.append(z)

                    DF_3dpositions = DF_3dpositions.append(pd.Series(coords,index = header), ignore_index = True)

                    num_frame += 1

    except RuntimeError:
        pass
    finally:
        pipeline.stop()


    #add time to 3d coordinates
    DF_3dpositions.insert(loc=1, column='Time_Stamp (s)', value=(np.array(time_st)-time_st[0])/1000)
    landmarks_3D_file = BAG_File[:-4]+'_Landmarks3D.csv'
    DF_3dpositions.to_csv(landmarks_3D_file)

    cvs_frame_info = BAG_File[:-4]+'_frameInfoDepth.csv'
    DF = pd.DataFrame()
    DF['Actual_Frame_Number'] = true_frame_number
    DF['Frame_Time_Stamp'] = (np.array(time_st)-time_st[0])/1000
    DF['Frame_Number_in_Video'] = frame_number
    print(cvs_frame_info)
    DF.to_csv(cvs_frame_info)



In [5]:
# get all files in path and process each one. Show progress with a bar
paths = [r'C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data']
#r'C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\example'

for path in paths:
    Files = os.listdir(path)            
    ext=('.bag')
    Files = [i for i in Files if i.endswith(tuple(ext))]
    Files_select = []
    for k,f in enumerate(Files):
        Files[k] = os.path.join(path,f)
        #if ('BIGSMILE' in f): 
        Files_select.append(Files[k])
           
    pbar = tqdm(total=len(Files_select))

    for f in Files_select:
        try: 
            get_color_from_bag(f)
            get_landmarks(f, face_detection_net, face_alignment_net)
            smooth_landmarks(f)
            get_3d_data(f)
        except:
            print('the file in exception is:')
            print(f)
            print()

        pbar.update(1)
    
# def do_work(f, models):
#     face_detection_net = models['FaceDetection']
#     face_alignment_net = models['FaceAlignment']
    
#     get_color_from_bag(f)
#     get_landmarks(f, face_detection_net, face_alignment_net)
#     smooth_landmarks(f)
#     get_3d_data(f)
#     pbar.update(1)


  0%|                                                                                           | 0/45 [00:00<?, ?it/s]

here1
here2
28.025989434201982
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\DJ01_02_BBP_NORMAL_color.mp4
here4
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\DJ01_02_BBP_NORMAL_frameInfoColor.csv
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\DJ01_02_BBP_NORMAL_landmarks.csv
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\DJ01_02_BBP_NORMAL_frameInfoDepth.csv


  2%|█▊                                                                                 | 1/45 [00:56<41:44, 56.93s/it]

here1
here2
26.341420461332316
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\DJ01_02_BBP_SLOW_color.mp4
here4
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\DJ01_02_BBP_SLOW_frameInfoColor.csv
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\DJ01_02_BBP_SLOW_landmarks.csv
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\DJ01_02_BBP_SLOW_frameInfoDepth.csv


  4%|███▋                                                                               | 2/45 [01:57<41:41, 58.16s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▎   | 43/45 [47:08<02:35, 77.78s/it]

here1
here2
29.29643154815528
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\RM4_02_NSM_SPREAD_NORMAL_color.mp4
here4
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\RM4_02_NSM_SPREAD_NORMAL_frameInfoColor.csv
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\RM4_02_NSM_SPREAD_NORMAL_landmarks.csv
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\RM4_02_NSM_SPREAD_NORMAL_frameInfoDepth.csv


 98%|████████████████████████████████████████████████████████████████████████████████▏ | 44/45 [47:53<01:08, 68.06s/it]

here1
here2
29.463569305579995
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\RM4_02_RST_REST_color.mp4
here4
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\RM4_02_RST_REST_frameInfoColor.csv
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\RM4_02_RST_REST_landmarks.csv
C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\deniz_data\RM4_02_RST_REST_frameInfoDepth.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [48:32<00:00, 59.28s/it]

In [6]:
Files_select

['C:\\Users\\GuarinD\\Documents\\GitHub\\Face_and_Gestures_2020\\deniz_data\\DJ01_02_BBP_NORMAL.bag',
 'C:\\Users\\GuarinD\\Documents\\GitHub\\Face_and_Gestures_2020\\deniz_data\\DJ01_02_BBP_SLOW.bag',
 'C:\\Users\\GuarinD\\Documents\\GitHub\\Face_and_Gestures_2020\\deniz_data\\DJ01_02_DDK_PA_FAST.bag',
 'C:\\Users\\GuarinD\\Documents\\GitHub\\Face_and_Gestures_2020\\deniz_data\\DJ01_02_DDK_PA_NORM.bag',
 'C:\\Users\\GuarinD\\Documents\\GitHub\\Face_and_Gestures_2020\\deniz_data\\DJ01_02_NSM_BIGSMILE_HOLD.bag',
 'C:\\Users\\GuarinD\\Documents\\GitHub\\Face_and_Gestures_2020\\deniz_data\\DJ01_02_NSM_BIGSMILE_NORM.bag',
 'C:\\Users\\GuarinD\\Documents\\GitHub\\Face_and_Gestures_2020\\deniz_data\\DJ01_02_NSM_OOEE_NORM.bag',
 'C:\\Users\\GuarinD\\Documents\\GitHub\\Face_and_Gestures_2020\\deniz_data\\DJ01_02_NSM_OOEE_SLOW.bag',
 'C:\\Users\\GuarinD\\Documents\\GitHub\\Face_and_Gestures_2020\\deniz_data\\DJ01_02_NSM_OPEN_DIS.bag',
 'C:\\Users\\GuarinD\\Documents\\GitHub\\Face_and_Gestures_2

In [7]:
#Smooth landmarks positions and generate a new video showing landmarks
if 0:
    BAG_File = r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\example\N_02_NSM_SPREAD.bag"
    LandmarkDataFrame = pd.read_csv(BAG_File[:-4]+'_landmarks.csv', index_col=0)
    windowlength=5
    for i in range(68):
        num=str(i)
        xx = LandmarkDataFrame['landmark_'+num+'_x'].values
        xx_med = signal.medfilt(xx,kernel_size=windowlength)

        yy = LandmarkDataFrame['landmark_'+num+'_y'].values
        yy_med = signal.medfilt(yy,kernel_size=windowlength)  

        LandmarkDataFrame['landmark_'+num+'_x'] = xx_med
        LandmarkDataFrame['landmark_'+num+'_y'] = yy_med

    landmark_file = BAG_File[:-4]+'_landmarksFiltered.csv'
    LandmarkDataFrame.to_csv(landmark_file)    

    color_file = BAG_File[:-4]+'_color.mp4'
    video_handler = cv2.VideoCapture(color_file)  # read the video
    FPS = video_handler.get(cv2.CAP_PROP_FPS)
    width = video_handler.get(3)   # float
    height = video_handler.get(4) # float

    color_file_landmark = BAG_File[:-4]+'_colorlandmark.mp4'
    video = VideoWriter(color_file_landmark, -1, int(FPS), (int(width),int(height)))

    success = True 
    k=0
    for k in range(len(LandmarkDataFrame)):
        success, image = video_handler.read()

        frame_number=k+1
        frame_information = LandmarkDataFrame.loc[LandmarkDataFrame['Video_Frame_number'] == frame_number].values
        shape = np.array([frame_information[0][7:]])
        shape = np.reshape(shape.astype(np.int), (-1, 2))
        bb = np.array([frame_information[0][3:7]])


        for j, (x, y) in enumerate(shape):
            if x is np.NaN:
                continue
            elif j<17:
                continue
            else:
                cv2.rectangle(image, (int(bb[0][0]), int(bb[0][1])), (int(bb[0][2]), int(bb[0][3])), (255,0,0), 1)
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)



        frame_to_save = image
        video.write(frame_to_save)
        k +=1

    video.release()
    print('done')

In [8]:
def getColorFromBag_newFolder(BAG_File, new_path=None):

    pipeline = rs.pipeline()
    config = rs.config()

    rs.config.enable_device_from_file(config, BAG_File, repeat_playback=False)

    config.enable_all_streams()
    profile = pipeline.start(config)

    # create alignment object
    align_to = rs.stream.color
    align = rs.align(align_to)

    # inform the device that this is not live streaming from camera
    playback = profile.get_device().as_playback()
    playback.set_real_time(False)
    duration = playback.get_duration()

    true_frame_number = []
    frame_number = []
    time_st = []

    num_frame = 0


    Color_Frames = []#{}

    try:
        while True:
            frames = pipeline.wait_for_frames()  #get frame from file 

            this_frame = frames.get_frame_number()  #get frame number 

            if (num_frame != 0) and (true_frame_number[-1] == this_frame): #verify that frame number is not repeated 
                #if frame number is repeated then replace the stored information 
                aligned_frames = align.process(frames)

                #take color and depth from frame, if any to these is not available then skip the frame
                aligned_depth = aligned_frames.get_depth_frame()
                aligned_color = aligned_frames.get_color_frame()

                # validate that both frames are available
                if not aligned_depth or not aligned_color:
                    continue

                time_stamp = frames.get_timestamp()
                true_frame_number[-1] = frames.get_frame_number()
                time_st[-1] = time_stamp 

                # transform to np array

                color_data = np.asanyarray(aligned_color.as_frame().get_data(), dtype=np.int)
                #depth_data = np.asanyarray(aligned_depth.as_frame().get_data(), dtype=np.int)
                # adjust depth data in meters
                #depth_data *= depth_scale

                Color_Frames[-1] = color_data

            else:
                #if frame number is not repeated then append the stored information 
                aligned_frames = align.process(frames)

                #take color and depth from frame, if any to these is not available then skip the frame
                aligned_depth = aligned_frames.get_depth_frame()
                aligned_color = aligned_frames.get_color_frame()

                # validate that both frames are available
                if not aligned_depth or not aligned_color:
                    continue

                time_stamp = frames.get_timestamp()
                true_frame_number.append(frames.get_frame_number())
                time_st.append(time_stamp )

                # transform to np array

                color_data = np.asanyarray(aligned_color.as_frame().get_data(), dtype=np.int)
                #depth_data = np.asanyarray(aligned_depth.as_frame().get_data(), dtype=np.int)
                # adjust depth data in meters
                #depth_data *= depth_scale

                Color_Frames.append(color_data)
                #Depth_Frames.append(depth_data

                frame_number.append(num_frame)
                num_frame += 1

    except RuntimeError:
        pass
    finally:
        pipeline.stop()
 

    try: 
        duration_movie = duration.total_seconds()
        FPS = num_frame/duration_movie
        height, width,_ =  Color_Frames[0].shape


        root, file_name = os.path.split(BAG_File)
        
        if new_path is not None:
            root = new_path

        print(root)
        color_file = os.path.join(root,file_name[:-4]+'_color.mp4')

        video = VideoWriter(color_file, 0, int(FPS), (width,height))

        for k in range(num_frame):
            frame_to_save = Color_Frames[k].astype('uint8')
            video.write(frame_to_save)

        video.release()    


        cvs_frame_info = os.path.join(root,file_name[:-4]+'_frameInfoColor.csv')
        df_cols = ['Actual_Frame_Number', 'Frame_Time_Stamp', 'Frame_Number_in_Video']
        df = pd.DataFrame(columns=df_cols)
        df['Actual_Frame_Number'] = true_frame_number
        df['Frame_Time_Stamp'] = (np.array(time_st)-time_st[0])/1000
        df['Frame_Number_in_Video'] = frame_number

        df.to_csv(cvs_frame_info)
    except:
        pass

In [9]:
if 0:
    Files_dataFrame = pd.read_csv(r"C:\Users\GuarinD\Documents\color_videos\files.csv")
    paths_to_save = [r'C:\Users\GuarinD\Documents\color_videos\PD', r'C:\Users\GuarinD\Documents\color_videos\NF']

    Files = Files_dataFrame['File_Name']

    getColorFromBag_newFolder(Files[7], new_path=paths_to_save[1])


    for k, file in enumerate(Files):
        if k < 24:
            continue 

        subject_ID = Files_dataFrame['Subject_ID'][k]
        if 'PD' in subject_ID:
            getColorFromBag_newFolder(file, new_path=paths_to_save[0])
        else:
            getColorFromBag_newFolder(file, new_path=paths_to_save[1])




In [10]:
#transfer files from one location to another based on name
